# CSS Lab: Networks
This lab demonstrates how to use common tools to analyze the structure of several networks. Topics covered include visualization, centrality measures, shortest paths, and affiliation networks. At the end of the lab, you will be able to visualze and perform an analysis of your own social network.

## Contents

1. [Setup](#Setup)
2. [Build and visualize a network](#Build-and-visualize-a-network)
    1. [Load the network](#Load-the-network)
    2. [Visualize the network](#Visualize-the-network)
3. [Centrality measures](#Centrality-measures)
    1. [Find centrality](#Find-centrality)
    2. [Interpret centrality](#Interpret-centrality)
    3. [Compare centrality measures](#Compare-centrality-measures)
7. [Your social network](#Your-social-network)
    1. [Visualize your social network](#Visualize-your-social-network)
    3. [Social network centrality](#Social-network-centrality)
8. [References](#References)
    


## 1. Setup
We import several libraries, including `networkx` for network algorithms, `pandas` for data processing, and `visJS2jupyter` for visualization. We also define some helper functions used by several of the 

In [ ]:
%pylab inline
import itertools
import json
import math
import urllib.request
import networkx as nx
import networkx as networkx
import networkx.algorithms as nxalg
import networkx.algorithms.community as nxcom
import networkx.readwrite as nxrw
import pandas as pd
import visJS2jupyter.visJS_module as vjs

In [ ]:
# Helper functions

# Standardize a vector to 0 mean and unit variance
def scale(v):
    return (v - v.mean()) / v.std()

# Return a data frame sorted by a node's centrality relative the mean of its centralities
def rel_centrality(df, measure, labels=dict()):
    name = 'rel_{}'.format(measure)
    return pd.DataFrame({
        name: scale(df[measure]) / df['mean_centrality'],
        "label": [labels.get(x, x) for x in df.index]
    })

def get_colors():
    phi = (1 + math.sqrt(5)) / 2
    color = []
    for i in range(1, 20):
        theta = phi * i * math.pi * 2
        x = 128 + math.floor(64*math.sin(theta))
        y = 128 + math.floor(64*math.cos(theta))
        color.append((x, x, y))
    return color

def visualize_visjs(
        G, communities=None, colors=None, default_color="192,192,192",
        node_size_field="node_size", layout="spring", scale=500, pos=None,
        groups=None, weight=None, labels=dict(), title=""):
    # Get list of nodes and edges
    nodes = list(G.nodes())
    edges = list(G.edges())
    # Change node shapes for bipartite graph
    if groups is None:
        node_shapes = dict()
        node_sizes = dict()
        node_colors = dict()
    else:
        node_shapes = dict((n, "square") for n in groups)
        node_sizes = dict((n, 15) for n in groups)
        node_colors = dict((n, "192,128,0") for n in groups)
    # Per-node properties
    nodes_dict = dict((n, {
        "id": labels.get(n, n),
        "node_size": node_sizes.get(n, 5),
        "node_shape": node_shapes.get(n, "dot")
        }) for n in nodes)
    # Generate a layout for the nodes
    edge_smooth_enabled = False
    edge_width = 4
    edge_arrow_scale = 2
    if communities is not None and pos is None:
        # Generate initial positions based on community
        phi = 3.14 / len(nodes)
        community_node = []
        # Create list of nodes and their communities
        for i, com in enumerate(sorted(communities, key=lambda x: len(x), reverse=True)):
            for node in com:
                community_node.append((i, node))
        # Sort by community and
        community_node = sorted(community_node)
        # Generate initial position by placing communities around a circle
        pos = dict((d[1], (math.cos(i*phi), math.sin(i*phi))) for i, d in enumerate(community_node))
    else:
        pos = None
    if layout == "circle":
        pos = nx.circular_layout(G, scale=scale)
    elif layout == "spring":
        pos = nx.spring_layout(G, k=3/math.sqrt(len(nodes)), scale=scale, pos=pos)
    else:
        edge_smooth_enabled = True
    # Assign position
    for n in nodes:
        nodes_dict[n]["x"] = pos[n][0]
        nodes_dict[n]["y"] = pos[n][1]
    # Calculate bounds for scaling
    x_min = min(pos.values(), key=lambda x: x[0])[0]
    x_max = max(pos.values(), key=lambda x: x[0])[0]
    y_min = min(pos.values(), key=lambda x: x[1])[1]
    y_max = max(pos.values(), key=lambda x: x[1])[1]
    x_range = x_max - x_min
    y_range = y_max - y_min
    max_range = max(x_range, y_range)
    # If we have communities, assign color based on community
    if colors is None:
        colors = ["{},{},{}".format(*c) for c in get_colors()]
    if communities is not None:
        for i, com in enumerate(sorted(communities, key=lambda x: len(x), reverse=True)):
            for node in com:
                try:
                    nodes_dict[node]["color"] = "rgba({},1)".format(colors[i])
                    nodes_dict[node]["color_index"] = i
                except IndexError:
                    nodes_dict[node]["color"] = "rgba({},1)".format(default_color)
    # Update color for bipartite nodes
    for node, node_attr in nodes_dict.items():
        if node in node_colors:
            node_attr["color"] = "rgba({},1)".format(node_colors[node])
    # Map node labels to contiguous ids
    node_map = dict(zip(nodes,range(len(nodes))))
    # Determine edge colors
    edge_colors_idx = {}
    for source, target in edges:
        source_color = nodes_dict[source].get("color_index", None)
        target_color = nodes_dict[target].get("color_index", None)
        if source_color == target_color and source_color is not None:
            edge_colors_idx[(source, target)] = source_color
    edge_colors = dict(
        (e,colors[c])
        for e, c in edge_colors_idx.items() if c < len(colors))
    # Per-edge properties, use contiguous ids to identify nodes
    edge_scale = math.ceil(max_range / 200)
    edges_dict = []
    for source, target, data in G.edges(data=True):
        edge = {
            "source": node_map[source],
            "target": node_map[target],
            "title":'test',
            "color": "rgba({},0.3)".format(edge_colors.get((source,target), default_color)),
            "edge_width_field": "value",
            "value": data.get("value", 1) * edge_scale
        }
        edges_dict.append(edge)
    # Convert nodes dict to node list
    nodes_list = [nodes_dict[n] for n in nodes]
    # Check for directed graph
    if G.__class__ == nx.classes.digraph.DiGraph:
        directed = True
    else:
        directed = False
    # Call visjs
    return vjs.visjs_network(
        nodes_list, edges_dict,
        node_size_field="node_size",
        node_size_multiplier=10.0,
        edge_width_field="value",
        edge_width=edge_width,
        edge_arrow_to=directed,
        edge_arrow_to_scale_factor=edge_arrow_scale,
        edge_smooth_enabled=edge_smooth_enabled,
        edge_smooth_type="curvedCW",
        graph_id=hash(title))


## 2. Build and visualize a network

This section loads network data from a file and explores its basic properties. 

In [ ]:
# Helper functions

def load_hamilton_affiliation():
    B = nxrw.adjlist.read_adjlist("external/hamilton.csv", delimiter="; ", comments="%")
    return B

def load_hamilton(threshold=None):
    # Load the song-character affiliation network
    B = load_hamilton_affiliation()
    # Get list of songs from the file
    songs = set()
    with open("external/hamilton.csv") as f:
        f.readline()
        for row in f:
            songs.add(row.split("; ")[0])
    # Deduce list of charactes
    characters = set(B.nodes()) - songs
    # Project the affiliation network onto the set of characters
    G = nxalg.bipartite.projection.weighted_projected_graph(B, characters)
    # Threshold
    if threshold is not None:
        for s, t, data in list(G.edges(data=True)):
            if data["weight"] < threshold:
                G.remove_edge(s,t)
    return G

### 2.1 Load the network

The next cell loads data from a file using the `networkx` library,
and displays a list of nodes in the network.
This example uses characters from the play _Hamilton_.

In [ ]:
G_hamilton = load_hamilton()
sorted(G_hamilton.nodes())

Now that you know the labels of the nodes, you can see which nodes are connected by an edge.
In this case, two nodes are connected by an edge if the corresponding characters have parts in at least one song together.
The next cell chooses a single node and prints a list of all the other nodes it's connected to.
These nodes are called its neighbors.

In [ ]:
sorted(G_hamilton.neighbors('E. Schuyler'))

### 2.2 Visualize the network
In these visualizations, each circle represents a node.
Edges between two nodes are represented by drawing a line between them.

There are many ways to draw a network.
One simple way is to space all the nodes evenly around a circle.

In [ ]:
visualize_visjs(G_hamilton, layout="circle", title="Circular Layout Example")

Another common way to visualize a network is using a "force-directed" layout.
In a force-directed layout, nodes push away from each other, but edges act like springs pulling them back together.
As a result, nodes with many neighbors in common are pulled closer to each other.

In [ ]:
visualize_visjs(G_hamilton, scale=2000, title="Force-Directed Layout Example")

### Short Answer Q1
In a few sentences, compare the benefits and drawbacks of circular and force-directed layouts. Consider large and small networks. Also consider which types of questions you can/can't answer with each layout.

(answer here ~150 words)

## 3. Centrality measures

One benefit of representing data as a network is that the patterns of connections between nodes can reveal useful information.
Many standard techniques for investigating the structure of networks have been developed.

One of the simplest questions to ask is: which nodes are most important?
But what does "important" mean exactly?
There are several common ways to measure importance, or _centrality_, of nodes in a nework.
This section examines several of the most popular.

While the _Hamilton_ network in the previous section is simple enough to be illustrative,
a historical data set better demonstrates how centrality measures can be applied to real-world networks.
This section uses historical data on affilations between organizers of the American Revolution,
taken from [Using Metadata to Find Paul Revere](https://kieranhealy.org/blog/archives/2013/06/09/using-metadata-to-find-paul-revere/).

In [ ]:
# Helper functions

def load_revere_affiliation(
        url="https://raw.githubusercontent.com/kjhealy/revere/master/data/PaulRevereAppD.csv"):
    data = urllib.request.urlopen(url)
    df = pd.read_csv(data).set_index("Unnamed: 0")
    people = list(df.index)
    groups = list(df.columns)
    #
    B = nx.Graph()
    for column in df.columns:
        for row in df[df[column] == 1].index:
            B.add_edge(column, row)
    return people, groups, B

def load_revere(
        url="https://raw.githubusercontent.com/kjhealy/revere/master/data/PaulRevereAppD.csv",
        threshold=None, dual=False):
    # Load the affiliation network
    people, groups, B = load_revere_affiliation(url)
    # Project the affiliation network onto the set of people
    if dual:
        G = nxalg.bipartite.projection.weighted_projected_graph(B, groups)
    else:
        G = nxalg.bipartite.projection.weighted_projected_graph(B, people)
    # Threshold
    if threshold is not None:
        for s, t, data in list(G.edges(data=True)):
            if data["weight"] < threshold:
                G.remove_edge(s,t)
    # Copy "weight" data to "value" needed by visjs
    w = nx.get_edge_attributes(G, "weight")
    nx.set_edge_attributes(G, w, "value")
    return G

### 3.1 Loading the network
The next cell begins by loading the data into a `networkx` graph. As is, an analysis of the data will reveal Paul Revere to be highly central by all measures [Healey2013] so to make the example more interesting, let's ask this: if Paul Revere had been on vacation, who might have notified the Massachusetts Provincial Congress of the impending attack in his place? To do so, we remove Paul Revere from the network. 

In [ ]:
G_revere = load_revere()
people, groups, B_revere = load_revere_affiliation()
G_revere.remove_node('Revere.Paul')

### 3.2 Visualizing the network
The next cell visualizes the network. You can zoom in and out and hover over nodes to see their labels.

In [ ]:
visualize_visjs(G_revere, scale=1500, title="American Revolution")

### Short answer  Q2
Look at the visualization above and write a few sentences about your intuition on who in this network might be influential. Do any nodes have more connections than others? Do any nodes act as bridges by connecting multiple communities? Are any nodes closely connected to the entire network? How might each of these three properties relate to an individuals role in a social movement such as the American Revolution?

(answer here ~150 words)

### 3.1 Find centrality
Now let's move on to examine the centralities. The next cell uses `networkx` to calculate node centralities and then stores them in a data frame. For each node we also calculate a mean centrality value.

In [ ]:
df = pd.DataFrame({"id": G.nodes(), "label": G.nodes()}).set_index("id")
df['degree'] = pd.Series(nx.degree_centrality(G))
df['betweenness'] = pd.Series(nx.betweenness_centrality(G))
df['closeness'] = pd.Series(nx.closeness_centrality(G))
df['eigenvector'] = pd.Series(nx.eigenvector_centrality(G))

In [ ]:
# Calculate mean of all centrality measures
df['mean_centrality'] = (
    scale(df['degree'])
    + scale(df['betweenness'])
    + scale(df['closeness'])
    + scale(df['eigenvector'])) / 4.0

#### 3.1.1 Degree

One very simple way to find important nodes is to count how many neighbors they have.
This measure is called the degree centrality.
This number is typically divided by the total number of other nodes in the network, so a value
of 0.82 means that a node is connected to 82% of the other nodes.
The next cell shows the nodes with the highest degree centralities.

Note that some of the people have the same degree. In fact, if two people have exactly the same set of neighbors,
all of their centrality scores will be the same.

In [ ]:
df.sort_values('degree', ascending=False).head(5)

__CUT THESE?__

Instead of searching for the highest degree centrality nodes, we can find nodes that have an unusually high degree centrality compared to its other centralities.

In [ ]:
rel_centrality(df, 'degree').sort_values('rel_degree', ascending=False).head()

#### 3.1.2 Betweenness

Rather than highly-connected nodes, you might want to find nodes that connect different parts of the network.
These types of nodes are sometimes called bridges, or brokers.
The betweenness centrality is based on finding the shortest path between nodes.
The nodes on that path play the role of bridges, connecting the endpoints.
So the betweenness is the fraction of all shortest paths in the network that pass through a given node.

In [ ]:
df.sort_values('betweenness', ascending=False).head(5)

In [ ]:
rel_centrality(df, 'betweenness').sort_values('rel_betweenness', ascending=False).head()

#### 3.1.3 Closeness

A nodes might also be considered important if it is close to many other nodes.
In other words, if the paths connecting it to other nodes are all very short.
This measure is called closeness centrality.

In [ ]:
df.sort_values('closeness', ascending=False).head(5)

In [ ]:
rel_centrality(df, 'closeness').sort_values('rel_closeness', ascending=False).head()

#### 3.1.4 Eigenvector

The last centrality measure covered in this section measures not just how well-connected a node is, but how well-connected its neighbors are as well.
This measure is the eigenvector centrality.
The PageRank algorithm used by Google to find high quality websites is an extension of this algorithm.

In [ ]:
df.sort_values('eigenvector', ascending=False).head(5)

In [ ]:
rel_centrality(df, 'eigenvector').sort_values('rel_eigenvector', ascending=False).head(5)

#### Short Answer Q3

Looking over the above centrality results, write a few sentences about how the different centrality measures compare. It may be helpful to see the list of organizations a person belonged to, which can be done in the next cell. Things to consider: which centrality measures give similar lists of individuals, whether the highest-ranked individuals make sense (and why), and how the lists of high-centrality individuals compare to your predictions from 

(answer ~150 words)

In [ ]:
list(nx.neighbors(B_revere, "Collier.Gershom"))

### Compare centrality measures

The following plot shows how different centrality measures are related to each other (in this network).
Each graph compares two centrality measures corresponding to its row and column.
In each graph, each point represents a single node in the network.

Points are drawn with transparency so overlapping points are darker.
If a group of people have the same centralities,
possibly because they have the same set of neighbors,
they appear as a single dark spot.

In [ ]:
# Store column names in array to keep order consistent
measures = ["degree", "betweenness", "eigenvector", "closeness"]
plt.figure(figsize=(7,6))
# Loop through rows and columns
# Even though we have 4 measures, we only need 3x3 to compare all
for row in range(3):
    for col in range(3):
        # Each row column pair only needs to be plotted once
        if row >= col:
            # The longest row should correspond to the measure that
            # does not appear on a column.
            x, y = df[measures[row]], df[measures[(col-1) % 4]]
            plt.subplot(3,3,1 + row*3 + col)
            plt.plot(x, y, '.', alpha=0.3, markersize=10)
            plt.xlim([0, x.max()]); plt.ylim([0, y.max()])
        if row == 2:
            plt.xlabel(measures[(col - 1) % 4])
        if col == 0:
            plt.ylabel(measures[row])
plt.tight_layout()

## Your social network
Now you can do these analyses on your own social network! Specifically, you can download a list of all of your friends and whether they are friends with each other.
This network is called your "ego network."

If you use Facebook, you can use the LostCircles plugin for Chrome to download your social network.
1. Open the following link in Chrome to install LostCircles:
https://chrome.google.com/webstore/detail/lost-circles-facebook-gra/ehpmfdlcppenimpibdifodjgfafkjhjl?hl=en
2. Click the LostCircles icon on the Chrome toolbar and select "Start Loading."
3. Wait until the menu shows "100%" next to "Start Loading."
4. Click the LostCircles icon and then "Download..." followed by "JSON (no pics)". Save the result as `external/LostCircles/egonet.json`.
5. Continue to the next cell

In [ ]:
# Helper functions

def load_lost_circles_json(in_file, name=False):
    with open(in_file) as f:
        raw = json.load(f)
    id_to_name = dict((i, datum["name"]) for i, datum in enumerate(raw['nodes']))
    if name:
        edges = [(id_to_name[datum["source"]], id_to_name[datum["target"]], {"capacity":1}) for datum in raw['links']]
    else:
        edges = [(datum["source"], datum["target"], {"capacity":1}) for datum in raw['links']]
    G = nx.Graph()
    G.add_edges_from(edges)
    return G, id_to_name, edges

### Visualize your social network
Let's start by visualizing your ego network.
* Does it look like there are distinct groups? If so, can you explain why those groups exist?
* Is everyone connected? Or are there disconnected components of the network?

In [ ]:
G, id_to_name, edges = load_lost_circles_json("external/LostCircles/egonet.json")
visualize_visjs(G, title="Ego Net", scale=6000, labels=id_to_name)

### Social network centrality
Now you can repeat the centrality analysis for members of your own ego network.

In [ ]:
# Calculate centrality measures
df = pd.DataFrame({"id": G.nodes(), "label": [id_to_name[x] for x in G.nodes()]}).set_index("id")
df['degree'] = pd.Series(nx.degree_centrality(G))
df['betweenness'] = pd.Series(nx.betweenness_centrality(G))
df['closeness'] = pd.Series(nx.closeness_centrality(G))
df['eigenvector'] = pd.Series(nx.eigenvector_centrality(G))
# Calculate mean of all centrality measures
df['mean_centrality'] = (
    scale(df['degree'])
    + scale(df['betweenness'])
    + scale(df['closeness'])
    + scale(df['eigenvector'])) / 4.0

#### Degree
These are the most highly connected members of your network. Do they have anything in common?

In [ ]:
df.sort_values('degree', ascending=False).head(5)

In [ ]:
rel_centrality(df, 'degree', labels=id_to_name).sort_values('rel_degree', ascending=False).head(5)

#### Betweenness
These are the individuals who connect different parts of your network. Can you think of reasons these particular individuals might play that role?

In [ ]:
df.sort_values('betweenness', ascending=False).head(5)

In [ ]:
rel_centrality(df, 'betweenness', labels=id_to_name).sort_values('rel_betweenness', ascending=False).head(5)

#### Closeness
These individuals are, on average, closest to everyone you know. Do the results make sense?

In [ ]:
df.sort_values('closeness', ascending=False).head(5)

In [ ]:
rel_centrality(df, 'closeness', labels=id_to_name).sort_values('rel_closeness', ascending=False).head(5)

#### Eigenvector
These individuals are most connected to highly connected individuals. How do they compare to those identified by other methods? Can you explain why these individuals would have high centrality?

In [ ]:
df.sort_values('eigenvector', ascending=False).head(5)

In [ ]:
rel_centrality(df, 'eigenvector', labels=id_to_name).sort_values('rel_eigenvector', ascending=False).head(5)

## Reflection Questions


* Did any of the measures show the same people as being most central?
* Members of the Tea Party (e.g. `Collier.Gershom`) have a high degree centrality, but low betweenness centrality. What does that tell you about the Tea Party as compared to other groups?
* What would a group with _high_ betweenness and _low_ degree centrality look like?
* Of the important people you identified from the visualization, were any in the lists of high-centrality individuals? If so, which types of centrality.

## References

[Healey2013] Healy, K. 2013. "Using Metadata to Find Paul Revere." https://kieranhealy.org/blog/archives/2013/06/09/using-metadata-to-find-paul-revere/

[CNM2004] Clauset, A., Newman, M. E., & Moore, C. (2004). Finding community structure in very large networks. Physical review E, 70(6).

[RLWGBF2017] Rosenthal, S. B., Len, J., Webster, M., Gary, A., Birmingham, A., & Fisch, K. M. (2017). Interactive network visualization in Jupyter notebooks: visJS2jupyter. Bioinformatics.

[BS2016] A. Beveridge and J. Shan, "Network of Thrones," Math Horizons Magazine, Vol. 23, No. 4 (2016), pp. 18-22

[Breiger1974] Breiger, R. L. (1974). The duality of persons and groups. Social forces, 53(2), 181-190.

[Granovetter1977] Granovetter, M. S. (1977). The strength of weak ties. In Social networks (pp. 347-367).